In [2]:
!pip3 install -q tensorflow-recommenders
!pip3 install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.0 MB/s eta 0:00:00


In [3]:
from typing import Dict, Text

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Ratings data.
ratings = tfds.load('movielens/latest-small-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/latest-small-ratings', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/100836 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/latest-small-ratings/0.1.1.incompleteOYSF54/movielens-train.tfre…

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/latest-small-ratings/0.1.1. Subsequent calls will reuse this data.


In [4]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)

In [5]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [6]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [7]:
import numpy as npi

# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

Epoch 1/3
25/25 [==============================] - 1388s 56s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 9.9171e-06 - factorized_top_k/top_10_categorical_accuracy: 3.9668e-05 - factorized_top_k/top_50_categorical_accuracy: 1.8842e-04 - factorized_top_k/top_100_categorical_accuracy: 4.4627e-04 - loss: 33380.0171 - regularization_loss: 0.0000e+00 - total_loss: 33380.0171
Epoch 2/3
25/25 [==============================] - 1364s 54s/step - factorized_top_k/top_1_categorical_accuracy: 7.9337e-05 - factorized_top_k/top_5_categorical_accuracy: 1.7851e-04 - factorized_top_k/top_10_categorical_accuracy: 3.9668e-04 - factorized_top_k/top_50_categorical_accuracy: 0.0024 - factorized_top_k/top_100_categorical_accuracy: 0.0066 - loss: 30663.4465 - regularization_loss: 0.0000e+00 - total_loss: 30663.4465
Epoch 3/3
25/25 [==============================] - 1311s 52s/step - factorized_top_k/top_1_categorical_accuracy: 6.9420e-04 - factor

In [14]:
import numpy as np

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["2"]))
print(f"Top 10 recommendations for user 2: {titles[:10]}")

Top 10 recommendations for user 2: [b'The Drop (2014)' b'The Drop (2014)' b'The Drop (2014)'
 b'Warrior (2011)' b'Warrior (2011)' b'Warrior (2011)' b'Warrior (2011)'
 b'Warrior (2011)' b'Warrior (2011)' b'Warrior (2011)']
